<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/Validate_on_JPEG_Quality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%time
import os
if not os.path.isdir('urban_oculus'):
  !git clone https://github.com/tgandor/urban_oculus
# first, this doesn't kill runtime
%run urban_oculus/snippets/get_coco_val2017_nounzip.ipynb
# this may kill runtime
%run urban_oculus/snippets/get_detectron2.ipynb

fatal: destination path 'urban_oculus' already exists and is not an empty directory.


ERROR:root:File `'urban_oculus/snippets/get_coco_val2017_nounzip.ipynb.py'` not found.


CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11 µs
CPU times: user 119 ms, sys: 90.9 ms, total: 210 ms
Wall time: 17.4 s


In [0]:
quality = 25

In [0]:
%%time
print('unpacking')
!unzip -o val2017.zip -d datasets/coco 2>&1 | tqdm --total 5000 >/dev/null
!du -h datasets/coco/val2017
print(f'degrading quality to {quality}')
!mogrify -verbose -quality {quality} datasets/coco/val2017/*.jpg 2>&1 | tqdm --total 10000 >/dev/null
!du -h datasets/coco/val2017

unpacking
5001it [00:06, 761.54it/s]              
788M	datasets/coco/val2017
degrading quality to 25
100% 10000/10000 [00:50<00:00, 198.18it/s]
132M	datasets/coco/val2017
CPU times: user 645 ms, sys: 233 ms, total: 878 ms
Wall time: 1min 8s


In [0]:
%%time
import detectron2.model_zoo
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
data_loader = build_detection_test_loader(cfg, 'coco_2017_val')
evaluator = COCOEvaluator('coco_2017_val', cfg, False)
results = inference_on_dataset(predictor.model, data_loader, evaluator)
results

Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.77s).
Accumulating evaluation results...
DONE (t=2.43s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.428
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.310
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.124
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.309
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.404
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.241
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.337
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.341
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [0]:
result_file = f'coco_val2017_rcnn_3x_q_{quality}.json'
import json

with open(result_file, 'w') as f:
  f.write(json.dumps(results['bbox']))

try:
  from google.colab import files
  files.download(result_file)
except ImportError:
  # running locally
  from IPython.core.display import display, HTML
  display(HTML(f'Results: <a href="{result_file}" target="_blank">{result_file}</a>'))